Для изменения количества тестовых данных и их разрядности следует изменить следующие две переменные ниже. Cледует задавать разрядную сетку с запасом, т.к тестовое окружение и скрипт не проверяют значение при ее переполнении, а сравнивают с корректным из-за чего возникают ошибки

In [ ]:
TEST_VECTORS_NUM = 10
BIT_WIDTH = 32

In [30]:
import os
import random

In [31]:
def update_tb_parameters(data_width, num_lines):
    with open("tb/top_python_tb.sv", "r") as file:
        lines = file.readlines()

    for i, line in enumerate(lines):
        if "localparam  DATA_WIDTH" in line:
            lines[i] = f'  localparam  DATA_WIDTH  = {data_width};\n'
        elif "localparam  NUM_LINES" in line:
            lines[i] = f'  localparam  NUM_LINES   = {num_lines};\n'

    with open("tb/top_python_tb.sv", "w") as file:
        file.writelines(lines)

In [32]:
def generate_mem_file(N=10, min_value=-100, max_value=100, bit_width=32):
  a_array = []
  b_array = []
  c_array = []
  d_array = []

  update_tb_parameters(bit_width,N)

  with open("input_data.mem", "w") as file:
      for i in range(N):
        a = random.randint(min_value, max_value)
        b = random.randint(min_value, max_value)
        c = random.randint(min_value, max_value)
        d = random.randint(min_value, max_value)

        a_bin = f"{a & (2**bit_width - 1):0{bit_width}b}"
        b_bin = f"{b & (2**bit_width - 1):0{bit_width}b}"
        c_bin = f"{c & (2**bit_width - 1):0{bit_width}b}"
        d_bin = f"{d & (2**bit_width - 1):0{bit_width}b}"

        file.write(f"{a_bin} {b_bin} {c_bin} {d_bin}\n")

        a_array.append(a)
        b_array.append(b)
        c_array.append(c)
        d_array.append(d)

  return a_array, b_array, c_array, d_array




In [33]:
def read_output_data(file_name="output_data.txt"):
    q_expected = []  
    q_real     = []  

    with open(file_name, "r") as file:
        for line in file:
            value1, value2 = [int(x) for x in line.split()]
            q_expected.append(value1)
            q_real.append(value2)

    return  q_expected, q_real

In [34]:
def calculate_and_compare(a_array, b_array, c_array, d_array, q_expected, q_real):
    errors = 0
    for i in range(len(a_array)-1):
        result = int(((a_array[i] - b_array[i]) * (1 + 3 * c_array[i]) - 4 * d_array[i]) // 2)

        if ((result != q_expected[i]) & (result == q_real[i])):
            print(f"Тест {i + 1}: Ошибка, рассчитанное в tb значение неверно! Результат: {result}, Ожидаемое из tb: {q_expected[i]}, Результат из модуля: {q_real[i]}")
            errors += 1
        elif ((result == q_expected[i]) & (result != q_real[i])):
            print(f"Тест {i + 1}: Ошибка, рассчитанное в модуле значение неверно! Результат: {result}, Ожидаемое из tb: {q_expected[i]}, Фактическое: {q_real[i]}")
            errors += 1
        elif ((result != q_expected[i]) & (result != q_real[i])):
            print(f"Тест {i + 1}: Ошибка, рассчитанное в модуле и tb значения неверно! Результат: {result}, Ожидаемое из tb: {q_expected[i]}, Фактическое: {q_real[i]}")
            errors += 1
    if (errors == 0):
      print("Тест прошел успешно")
    else:
      print(f"Обнаружено {errors} ошибок")



Ниже происходит генерация входных данных в входной файл для testbench

In [35]:
a_array, b_array, c_array, d_array = generate_mem_file(N=TEST_VECTORS_NUM,bit_width=BIT_WIDTH)



Следующий скрипт стоит запускать после запуска симуляции в Vivado, т.к он считывает файл являющийся результатом работы tb

In [36]:
q_expected, q_expected = read_output_data()
calculate_and_compare(a_array,b_array,c_array,d_array,q_expected,q_expected)

Тест 1: Ошибка, рассчитанное в модуле и tb значения неверно! Результат: 18740, Ожидаемое из tb: 255, Фактическое: 255
Тест 2: Ошибка, рассчитанное в модуле и tb значения неверно! Результат: -382, Ожидаемое из tb: 130, Фактическое: 130
Тест 3: Ошибка, рассчитанное в модуле и tb значения неверно! Результат: -5942, Ожидаемое из tb: -256, Фактическое: -256
Тест 4: Ошибка, рассчитанное в модуле и tb значения неверно! Результат: -4922, Ожидаемое из tb: -218, Фактическое: -218
Тест 5: Ошибка, рассчитанное в модуле и tb значения неверно! Результат: -9522, Ожидаемое из tb: -256, Фактическое: -256
Тест 6: Ошибка, рассчитанное в модуле и tb значения неверно! Результат: -10042, Ожидаемое из tb: -178, Фактическое: -178
Тест 7: Ошибка, рассчитанное в модуле и tb значения неверно! Результат: 749, Ожидаемое из tb: 255, Фактическое: 255
Тест 8: Ошибка, рассчитанное в модуле и tb значения неверно! Результат: -890, Ожидаемое из tb: -58, Фактическое: -58
Тест 9: Ошибка, рассчитанное в модуле и tb значения